# Анализ данных телекоммуникаций

## Importing Libraries

In [1]:
import pandas as pd, pyarrow
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from pathlib import Path
from datetime import datetime

import re
# --- Настройка путей и sys.path ---
# Добавляем корневую директорию проекта в sys.path для импорта кастомных модулей
PROJECT_ROOT = Path().cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.config import config
from src.logger import logger
from src.database import clickhouse_engine, postgres_engine, ipdr_engine    


2025-10-27 16:16:03,890 | my_logger - INFO - ✅ PostgreSQL engine создан | /data/aturov/scoring/src/database.py:21


Configuration loaded successfully.


2025-10-27 16:16:03,930 | my_logger - INFO - ✅ ClickHouse engine создан | /data/aturov/scoring/src/database.py:36
2025-10-27 16:16:03,931 | my_logger - INFO - ✅ IPDR ClickHouse engine создан | /data/aturov/scoring/src/database.py:46


In [2]:
OVERDUE_DAYS_MAX = 30
TOTAL_OVERDUE = 60
CURRENT_DATE = datetime.now().date()
DATE_START = pd.to_datetime('2022-01-01').date()

## Данные

### Данные о кредитах

In [3]:
df_banking = pd.read_parquet(f"{config.environment.data_processed_path}/credit_data_031025.parquet", engine="pyarrow")
#df_banking = df_banking.query(f'contract_date_open >= "{DATE_START}"')
#df_banking = df_banking.query('name_code_credit == "Потребительские кредиты"')
df_banking.shape

(73957, 30)

In [4]:
df_banking.summa.describe()

count      73957.000000
mean      234411.423435
std       243668.711554
min            1.000000
25%        50000.000000
50%       150000.000000
75%       380097.000000
max      1000000.000000
Name: summa, dtype: float64

In [5]:
df_numbers = df_banking.sort_values(by=['inn', 'contract_date_open'], ascending=True)
df_numbers.drop_duplicates(subset=['inn', 'contract_date_open', 'id_credit'], keep='first', inplace=True)
df_numbers = df_numbers[['inn', 'array_phones']]
df_numbers

,inn,array_phones
0,10101194902445,"[""996772529356""]"
1,10101195302746,"[""996700205385""]"
2,10101195302746,"[""996700205385""]"
3,10101195400720,"[""996704608121""]"
4,10101195400720,"[""996704608121""]"
...,...,...
70753,23112200300806,"[""996507191045""]"
49184,23112200300818,"[""996999335503""]"
58197,23112200400145,"[""996501402906""]"
60237,23112200400145,"[""996501402906""]"


In [6]:
gr_cred = df_banking.groupby(['inn', 'status', 'contract_date_open'])['id_credit'].count().reset_index()
gr_cred.sort_values('contract_date_open', ascending=False).head(5)

,inn,status,contract_date_open,id_credit
12551,11403197301092,Отказано,2025-10-14,1
45165,21210199300953,Отказано,2025-10-13,1
53138,21808200400702,Отказано,2025-10-13,1
16831,11811200250233,Отказано,2025-10-13,1
16334,11804200000154,Отказано,2025-10-13,1


In [7]:
gr_cred

,inn,status,contract_date_open,id_credit
0,10101194902445,Одобрено,2023-03-29,1
1,10101195302746,Одобрено,2022-08-25,2
2,10101195400720,Одобрено,2023-04-12,2
3,10101195400894,Одобрено,2022-05-24,1
4,10101195400894,Одобрено,2024-07-04,1
...,...,...,...,...
70433,23112200300806,Отказано,2025-08-28,1
70434,23112200300818,Одобрено,2025-01-22,1
70435,23112200400145,Отказано,2025-05-26,1
70436,23112200400145,Отказано,2025-06-06,1


In [8]:
cond = (df_banking['overdue_max'] >= OVERDUE_DAYS_MAX) | (df_banking['total_overdue'] >= TOTAL_OVERDUE) | (df_banking['status'].isin(['Отказано']))
df_banking['target'] = cond.astype(bool)
df_banking.value_counts('target')


target
False    46922
True     27035
Name: count, dtype: int64

In [9]:
client_target_with_phone = df_banking[['inn', 'array_phones', 'target']]
client_target_with_phone

,inn,array_phones,target
0,10101194902445,"[""996772529356""]",False
1,10101195302746,"[""996700205385""]",False
2,10101195302746,"[""996700205385""]",False
3,10101195400720,"[""996704608121""]",False
4,10101195400720,"[""996704608121""]",False
...,...,...,...
73952,11811200250233,"[""996704880485""]",True
73953,21210199300953,"[""996772120213""]",True
73954,21808200400702,"[""996500997970""]",True
73955,11804200000154,"[""996708322829""]",True


## Данные телеком-компании

In [10]:
sql = """ 
select * from dict.dict_cont_status dcs
"""

df_dict = pd.read_sql(sql, clickhouse_engine)
df_dict.to_csv(f"{config.environment.data_raw_path}/dict_cont_status_{CURRENT_DATE}.csv", index=False)
df_dict.head()

,id,status
0,0,Terminated
1,1,Active
2,4,Suspended
3,3,Idle
4,2,New


### Найдем по номеру телефона


In [15]:
## найдем ближайщие пользователей по номеру телефона и дате открытия кредита в ClickHouse
sql = f"""
    SELECT
		c_flat.id_request,
		c_flat.contract_date_open as date_open,
		c_flat.id_credit,
		c_flat.inn as inn_eldik,
		c_flat.phone_eldik,
		s.subscription_id,
		s.subs_msisdn as phone_beeline,
		s.eff_dt AS subs_eff_dt,
		decrypt_vault(s.cust_individual_tax_number) AS inn_beeline,
		c_flat.sex,
		c_flat.date_birth,
		c_flat.birthplace,
		c_flat.marital_status,
		c_flat.summa,
		c_flat.interest_on_credit,
		c_flat.contract_length,
		c_flat.prev_credit_count,
		c_flat.sum_of_prev_credits,
		c_flat.overdue_max,
		c_flat.total_overdue,
		c_flat.status
FROM
	DWH.hfct_subs AS s
    GLOBAL
INNER JOIN (
	SELECT
		id as id_request,
		contract_date_open,
		id_credit,
		inn,
		arrayJoin(array_phones) AS phone_eldik,
		sex,
		date_birth,
		birthplace,
		marital_status,
		summa,
		interest_on_credit,
		contract_length,
		prev_credit_count,
		sum_of_prev_credits,
		overdue_max,
		total_overdue,
		status
	FROM
		data_science.credits_eldik
    ) AS c_flat
    ON
	s.subs_msisdn = c_flat.phone_eldik
	AND toDate(c_flat.contract_date_open) = toDate(s.eff_dt + INTERVAL 1 DAY)
    and contract_status = 1
   """
#df_subs = pd.read_sql(sql, clickhouse_engine)
#df_subs.to_csv(f"{config.environment.data_raw_path}/df_subs_phone_{CURRENT_DATE}.csv", index=False)
df_subs_phone = pd.read_csv(f"{config.environment.data_raw_path}/df_subs_phone_2025-10-27.csv")
logger.info(f"df_subs_phone shape: {df_subs_phone['id_request'].nunique()}")
df_subs_phone.rename(columns={'contract_date_open': 'date_open'}, inplace=True)
df_subs_phone['date_open'] = pd.to_datetime(df_subs_phone['date_open'], errors='coerce')
df_subs_phone['subs_eff_dt'] = pd.to_datetime(df_subs_phone['subs_eff_dt'], errors='coerce').dt.date
df_subs_phone['inn_eldik'] = pd.to_numeric(df_subs_phone['inn_eldik'], errors='coerce')
df_subs_phone['inn_beeline'] = pd.to_numeric(df_subs_phone['inn_beeline'], errors='coerce')
df_subs_phone[['phone_beeline', 'phone_eldik']] = df_subs_phone[['phone_beeline', 'phone_eldik']].astype('Int64')
df_subs_phone[['inn_eldik', 'inn_beeline']] = df_subs_phone[['inn_eldik', 'inn_beeline']].astype('Int64')


df_subs_phone.dropna(subset=['subs_eff_dt', 'date_open', 'id_credit'], inplace=True)
logger.info(f"df_subs_phone shape after dropna: {df_subs_phone['id_request'].nunique()}")
df_subs_phone = df_subs_phone[df_subs_phone['date_open'] >= pd.to_datetime(DATE_START)]
df_subs_phone = df_subs_phone.query('date_open > subs_eff_dt')
logger.info(f"df_subs_phone shape: {df_subs_phone['id_request'].nunique()}")
df_subs_phone.head()


2025-10-27 16:26:51,195 | my_logger - INFO - df_subs_phone shape: 15312 | /tmp/ipykernel_4131096/216288947.py:58
2025-10-27 16:26:51,224 | my_logger - INFO - df_subs_phone shape after dropna: 15312 | /tmp/ipykernel_4131096/216288947.py:69
2025-10-27 16:26:51,238 | my_logger - INFO - df_subs_phone shape: 14484 | /tmp/ipykernel_4131096/216288947.py:72


,id_request,date_open,id_credit,inn_eldik,phone_eldik,subscription_id,phone_beeline,subs_eff_dt,inn_beeline,sex,...,birthplace,marital_status,summa,interest_on_credit,contract_length,prev_credit_count,sum_of_prev_credits,overdue_max,total_overdue,status
0,d4baf743-c965-4044-8270-5f2e3dfe9e9f,2025-01-21,5900000000325,22706198300070,996220041104,233538,996220041104,2025-01-20,22611198800562,Мужчина,...,NaN,Женат(Замужем),950000,16.0,36,0,0,0,0,Одобрено
1,c0c58c08-887e-4bbb-b24d-732ba8dd1077,2024-08-22,3000000002354,23105199600680,996779949455,341787,996779949455,2024-08-21,<NA>,Мужчина,...,NaN,Женат(Замужем),120000,22.0,24,0,0,0,0,Одобрено
2,1ffa19d1-6c3a-4e4c-8d99-1c7c666d3d8d,2024-03-11,4600000003326,22210198000208,996777778762,450378,996777778762,2024-03-10,<NA>,Мужчина,...,NaN,Женат(Замужем),500000,6.0,36,0,0,0,0,Одобрено
3,a6e41272-7de8-4b2c-933b-a8cef534c047,2024-03-15,4900000002156,21801198400874,996771717273,444532,996771717273,2024-03-14,<NA>,Мужчина,...,NaN,Женат(Замужем),110000,24.0,12,0,0,0,0,Одобрено
4,3747edcf-fb29-4fba-9a1e-d21044f74c3a,2024-03-15,4100000002876,10101195704030,996222609999,666113,996222609999,2024-03-14,<NA>,Женщина,...,NaN,Вдовец/Вдова,1000000,0.0,36,0,0,0,0,Одобрено


In [16]:
df_subs_phone['id_request'].shape, df_subs_phone['id_credit'].nunique()
# Разница между количеством заявок и уникальных кредитов говорит о том, что по одному кредиту может быть несколько пользователей с таким телефоном в базе Beeline.


((14705,), 14484)

In [17]:
df_credits_id = df_subs_phone.copy()
logger.info(f"df_credits_id shape: {df_credits_id['id_credit'].nunique()}")
df_credits_id = df_credits_id[(pd.to_datetime(df_credits_id['date_open']) - pd.to_datetime(df_credits_id['subs_eff_dt'])) < pd.Timedelta(days=8)]
df_credits_id['match_phone'] = True
df_credits_id['match_inn'] = df_credits_id['inn_eldik'] == df_credits_id['inn_beeline']
df_credits_id['match_inn'].fillna(False, inplace=True)
logger.info(f"df_credits_id shape: {df_credits_id['id_credit'].nunique()}")

df_credits_id.to_csv(f"{config.environment.data_processed_path}/df_match_phone_{CURRENT_DATE}.csv", index=False)
logger.info(f"df_credits_id shape: {df_credits_id['id_credit'].nunique()}")


2025-10-27 16:26:55,674 | my_logger - INFO - df_credits_id shape: 14484 | /tmp/ipykernel_4131096/3661099322.py:2
/tmp/ipykernel_4131096/3661099322.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_credits_id['match_inn'].fillna(False, inplace=True)
2025-10-27 16:26:55,693 | my_logger - INFO - df_credits_id shape: 14484 | /tmp/ipykernel_4131096/3661099322.py:7
2025-10-27 16:26:55,804 | my_logger - INFO - df_credits_id shape: 14484 | /tmp/ipykernel_4131096/3661099322.py:10


In [18]:
df_credits_id.columns

Index(['id_request', 'date_open', 'id_credit', 'inn_eldik', 'phone_eldik',
       'subscription_id', 'phone_beeline', 'subs_eff_dt', 'inn_beeline', 'sex',
       'date_birth', 'birthplace', 'marital_status', 'summa',
       'interest_on_credit', 'contract_length', 'prev_credit_count',
       'sum_of_prev_credits', 'overdue_max', 'total_overdue', 'status',
       'match_phone', 'match_inn'],
      dtype='object')

In [20]:
COLUMNS_TO_USE = ['id_request', 'date_open', 'id_credit', 'inn_eldik', 'phone_eldik',
       'subscription_id', 'phone_beeline', 'subs_eff_dt', 'inn_beeline', 'sex',
       'date_birth', 'birthplace', 'marital_status',
        'summa', 'interest_on_credit', 'contract_length',
       'prev_credit_count', 'sum_of_prev_credits', 'overdue_max',
       'total_overdue', 'status', 'match_phone', 'match_inn']

In [21]:
df_credits_id_to_table = df_credits_id[COLUMNS_TO_USE].copy()
df_credits_id_to_table['date_open'] = (
    pd.to_datetime(df_credits_id_to_table['date_open'], errors='coerce')
    .dt.floor('D') 
)

### Найдем по ИНН

In [25]:
## найдем ближайщие пользователей по номеру телефона и дате открытия кредита в ClickHouse
sql = f"""
    SELECT
		c_flat.id_request,
		c_flat.contract_date_open as date_open,
		c_flat.id_credit,
		c_flat.inn as inn_eldik,
		c_flat.phone_eldik,
		s.subscription_id,
		s.subs_msisdn as phone_beeline,
		s.eff_dt AS subs_eff_dt,
		decrypt_vault(s.cust_individual_tax_number) AS inn_beeline,
		c_flat.sex,
		c_flat.date_birth,
		c_flat.birthplace,
		c_flat.marital_status,
		c_flat.summa,
		c_flat.interest_on_credit,
		c_flat.contract_length,
		c_flat.prev_credit_count,
		c_flat.sum_of_prev_credits,
		c_flat.overdue_max,
		c_flat.total_overdue,
		c_flat.status
FROM
	DWH.hfct_subs AS s
    GLOBAL
INNER JOIN (
	SELECT
		id as id_request,
		contract_date_open,
		id_credit,
		inn,
		arrayElement(array_phones, 1) AS phone_eldik,  -- первый номер из массива
		sex,
		date_birth,
		birthplace,
		marital_status,
		summa,
		interest_on_credit,
		contract_length,
		prev_credit_count,
		sum_of_prev_credits,
		overdue_max,
		total_overdue,
		status
	FROM
		data_science.credits_eldik
    ) AS c_flat
    ON c_flat.inn = toInt64OrNull(decrypt_vault(s.cust_individual_tax_number)) 
    AND toDate(c_flat.contract_date_open) = toDate(s.eff_dt + INTERVAL 1 DAY)
    AND contract_status = 1
    """
#df_subs_inn = pd.read_sql(sql, clickhouse_engine)
#df_subs_inn.to_csv(f"{config.environment.data_raw_path}/df_subs_inn_{CURRENT_DATE}.csv", index=False)
df_subs_inn = pd.read_csv(f"{config.environment.data_raw_path}/df_subs_inn_2025-10-27.csv")
logger.info(f"id_credit shape: {df_subs_inn['id_request'].nunique()}")
# у нас дубликаты могут быть по нескольким номерам на один инн
df_subs_inn.drop_duplicates(subset=['id_request', 'id_credit', 'subscription_id', 'date_open'], keep='first', inplace=True)
logger.info(f"id_credit shape: {df_subs_inn['id_request'].nunique()}")
df_subs_inn['date_open'] = pd.to_datetime(df_subs_inn['date_open'], errors='coerce')
df_subs_inn['subs_eff_dt'] = pd.to_datetime(df_subs_inn['subs_eff_dt'], errors='coerce').dt.date
df_subs_inn['inn_eldik'] = pd.to_numeric(df_subs_inn['inn_eldik'], errors='coerce')
df_subs_inn['inn_beeline'] = pd.to_numeric(df_subs_inn['inn_beeline'], errors='coerce')
df_subs_inn.dropna(subset=['subs_eff_dt', 'date_open', 'id_credit'], inplace=True)
logger.info(f"df_subs_inn shape after dropna: {df_subs_inn['id_request'].nunique()}")
df_subs_inn = df_subs_inn[df_subs_inn['date_open'] >= pd.to_datetime(DATE_START)]
df_subs_inn = df_subs_inn.query('date_open > subs_eff_dt')
logger.info(f"df_subs_inn shape: {df_subs_inn['id_request'].nunique()}")
df_subs_inn.head()


2025-10-27 16:39:14,988 | my_logger - INFO - id_credit shape: 11014 | /tmp/ipykernel_4131096/95877607.py:57
2025-10-27 16:39:15,000 | my_logger - INFO - id_credit shape: 11014 | /tmp/ipykernel_4131096/95877607.py:60
2025-10-27 16:39:15,021 | my_logger - INFO - df_subs_inn shape after dropna: 11014 | /tmp/ipykernel_4131096/95877607.py:66
2025-10-27 16:39:15,034 | my_logger - INFO - df_subs_inn shape: 10799 | /tmp/ipykernel_4131096/95877607.py:69


,id_request,date_open,id_credit,inn_eldik,phone_eldik,subscription_id,phone_beeline,subs_eff_dt,inn_beeline,sex,...,birthplace,marital_status,summa,interest_on_credit,contract_length,prev_credit_count,sum_of_prev_credits,overdue_max,total_overdue,status
0,b17f6d4c-0d0f-4b77-87dc-62a8cc6578ee,2025-04-25,680af2124289d31b16f123d7,10302199101508,9.967714e+11,12832378,996223575127,2025-04-24,10302199101508,Женщина,...,NaN,Женат(Замужем),100000,17.0,24,0,0,0,0,Отказано
1,3aefe965-7e0b-48e2-bb72-87bc4c40311b,2025-04-25,680b99f0ac46edbf44cb17fa,21511198901606,9.965010e+11,13872421,996227809030,2025-04-24,21511198901606,Мужчина,...,NaN,Холост(а),50000,17.0,12,0,0,0,0,Отказано
2,f651d3a2-d142-43b0-89ea-f94998bc66a0,2023-08-02,1400000001563,21711197000207,9.967041e+11,606876,996778391968,2023-08-01,21711197000207,Мужчина,...,NaN,Женат(Замужем),250000,21.0,36,0,0,0,0,Одобрено
3,507a6011-2aa5-4681-89d4-e382e2e5447e,2024-05-27,5000000002124,20205197100762,9.965046e+11,582319,996771574304,2024-05-26,20205197100762,Мужчина,...,NaN,Женат(Замужем),150000,10.0,24,0,0,0,0,Одобрено
4,6255a3f0-3cca-4c7f-b978-1402006a52f3,2025-04-25,680b41b47548c6a1bc8cde89,22309200600826,9.962211e+11,14734900,996771040045,2025-04-24,22309200600826,Мужчина,...,NaN,Холост(а),10000,17.0,20,0,0,0,0,Отказано


In [26]:
data_without_duplicates = df_subs_inn.copy()
# и не должен быть уже в выборке по телефону
data_without_duplicates = data_without_duplicates[~data_without_duplicates['id_request'].isin(df_subs_phone['id_request'])]
logger.info(f"data_without_duplicates shape: {data_without_duplicates['id_request'].nunique()}")

2025-10-27 16:39:24,254 | my_logger - INFO - data_without_duplicates shape: 5019 | /tmp/ipykernel_4131096/1062410031.py:4


In [27]:
only_one_number = data_without_duplicates.groupby('id_request')['subscription_id'].nunique().reset_index()
only_one_number = only_one_number[only_one_number['subscription_id'] == 1]
only_one = data_without_duplicates[data_without_duplicates['id_request'].isin(only_one_number['id_request'])]
logger.info(f"only_one shape: {only_one['id_request'].nunique()}")

2025-10-27 16:39:25,422 | my_logger - INFO - only_one shape: 3926 | /tmp/ipykernel_4131096/3796690464.py:4


In [28]:
only_one[['phone_beeline', 'phone_eldik']] = only_one[['phone_beeline', 'phone_eldik']].astype('Int64')
only_one[['inn_eldik', 'inn_beeline']] = only_one[['inn_eldik', 'inn_beeline']].astype('Int64')

only_one['match_phone'] = False
only_one['match_inn'] = True
logger.info(f"only_one shape: {only_one['id_request'].nunique()}")

only_one.to_csv(f"{config.environment.data_processed_path}/df_match_inn_{CURRENT_DATE}.csv", index=False)
logger.info(f"only_one shape: {only_one['id_request'].nunique()}")

/tmp/ipykernel_4131096/18295944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_one[['phone_beeline', 'phone_eldik']] = only_one[['phone_beeline', 'phone_eldik']].astype('Int64')
/tmp/ipykernel_4131096/18295944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_one[['inn_eldik', 'inn_beeline']] = only_one[['inn_eldik', 'inn_beeline']].astype('Int64')
/tmp/ipykernel_4131096/18295944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [29]:
df_subs_inn_to_table = only_one[COLUMNS_TO_USE].copy()

df_subs_inn_to_table['date_open'] = (
    pd.to_datetime(df_subs_inn_to_table['date_open'], errors='coerce')
    .dt.floor('D') 
)

### Объединим результаты и вставим в основную таблицу

In [30]:
df_credits_id_to_table.columns

Index(['id_request', 'date_open', 'id_credit', 'inn_eldik', 'phone_eldik',
       'subscription_id', 'phone_beeline', 'subs_eff_dt', 'inn_beeline', 'sex',
       'date_birth', 'birthplace', 'marital_status', 'summa',
       'interest_on_credit', 'contract_length', 'prev_credit_count',
       'sum_of_prev_credits', 'overdue_max', 'total_overdue', 'status',
       'match_phone', 'match_inn'],
      dtype='object')

In [31]:
df_subs_inn_to_table.columns

Index(['id_request', 'date_open', 'id_credit', 'inn_eldik', 'phone_eldik',
       'subscription_id', 'phone_beeline', 'subs_eff_dt', 'inn_beeline', 'sex',
       'date_birth', 'birthplace', 'marital_status', 'summa',
       'interest_on_credit', 'contract_length', 'prev_credit_count',
       'sum_of_prev_credits', 'overdue_max', 'total_overdue', 'status',
       'match_phone', 'match_inn'],
      dtype='object')

In [32]:
df_to_table = pd.concat([df_credits_id_to_table, df_subs_inn_to_table], axis=0)
df_to_table.shape

(18631, 23)

In [33]:
df_to_table.drop_duplicates(inplace=True)
df_to_table.shape

(18631, 23)

In [34]:
df_to_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18631 entries, 0 to 18255
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_request           18631 non-null  object        
 1   date_open            18631 non-null  datetime64[ns]
 2   id_credit            18631 non-null  object        
 3   inn_eldik            18631 non-null  Int64         
 4   phone_eldik          18619 non-null  Int64         
 5   subscription_id      18631 non-null  int64         
 6   phone_beeline        18631 non-null  Int64         
 7   subs_eff_dt          18631 non-null  object        
 8   inn_beeline          10349 non-null  Int64         
 9   sex                  18631 non-null  object        
 10  date_birth           18631 non-null  object        
 11  birthplace           215 non-null    object        
 12  marital_status       18631 non-null  object        
 13  summa                18631 non-null 

In [35]:
# сопоставление pandas dtype -> ClickHouse
dtype_map = {
    "int64": "Int64",
    "float64": "Float64",
    "datetime64[ns]": "Date",
    "object": "String",
    "bool": "UInt8",
    "boolean": "UInt8",
}
cols = []
for col, dt in df_credits_id_to_table.dtypes.items():
    ch_type = dtype_map.get(str(dt), "String")
    cols.append(f"`{col}` {ch_type}")

from sqlalchemy import text
from sqlalchemy.exc import SQLAlchemyError
# параметры таблицы
target_db = clickhouse_engine.url.database
target_table = "credits_subs_eldik_clean"
target_db = 'data_science'

cols_ddl = ",\n  ".join(cols)
ddl = f"""
CREATE TABLE IF NOT EXISTS {target_db}.{target_table} (
  {cols_ddl}
) ENGINE = MergeTree()
ORDER BY (date_open,id_request, id_credit)
"""

# создать таблицу
try:
    with clickhouse_engine.connect() as conn:
        conn.execute(text(ddl))
    logger.info(f"Created/checked table {target_db}.{target_table}")
except SQLAlchemyError as e:
    logger.error(f"Failed to create table: {e}")
    raise
# загрузка данных пакетами через pandas.to_sql (использует clickhouse-sqlalchemy dialect)
try:
    # если таблица большая — уменьшите chunksize
    df_to_table.to_sql(
        name=target_table,
        con=clickhouse_engine,
        schema=target_db,
        if_exists="append",
        index=False,
        chunksize=5000,
        method="multi"
    )
    logger.info("Upload to ClickHouse finished")
except Exception as e:
    logger.error(f"Failed to upload dataframe via clickhouse_engine: {e}")
    raise

2025-10-27 16:39:35,998 | my_logger - INFO - Created/checked table data_science.credits_subs_eldik_clean | /tmp/ipykernel_4131096/3309748782.py:34
2025-10-27 16:39:43,631 | my_logger - INFO - Upload to ClickHouse finished | /tmp/ipykernel_4131096/3309748782.py:50
